# The Vanishing/Exploding Gradients Problems
 * The vanishing gradients problem occurs when the gradients of the loss function with respect to the parameters become extremely small during backpropagation.
 * Conversely, the exploding gradients problem arises when the gradients of the loss function grow exponentially as they propagate backward through the layers during backpropagation.


## Weight Initialization
 * Proper initialization of network weights can help alleviate both vanishing and exploding gradients problems. 
 * Techniques like __Xavier__ initialization and __He__ initialization are commonly used to ensure that the weights are initialized in a way that keeps the signal propagated through the network.

In [1]:
# Importing essential package
import numpy as np
import tensorflow as tf
from tensorflow import keras

# List comprehension to get the names of initializers in Keras without underscores
initializer_names = [name for name in dir(keras.initializers) if not name.startswith("_")]

# Printing the initializer names
print(initializer_names)

['Constant', 'GlorotNormal', 'GlorotUniform', 'HeNormal', 'HeUniform', 'Identity', 'Initializer', 'LecunNormal', 'LecunUniform', 'Ones', 'Orthogonal', 'RandomNormal', 'RandomUniform', 'TruncatedNormal', 'VarianceScaling', 'Zeros', 'constant', 'deserialize', 'get', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform', 'identity', 'lecun_normal', 'lecun_uniform', 'ones', 'orthogonal', 'random_normal', 'random_uniform', 'serialize', 'truncated_normal', 'variance_scaling', 'zeros']


* Creating a dense layer with built-in initializers

In [2]:
# Creating a dense layer with 10 units, ReLU activation, and He normal weight initialization
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

* Defining an initializer with Variance Scaling (custom kernel initializer)

In [3]:
# Defining an initializer with Variance Scaling
init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg', distribution='uniform')

# Creating a dense layer with 10 units, ReLU activation, and custom kernel initializer
dense_layer = keras.layers.Dense(10, activation="relu", kernel_initializer=init)

## Nonsaturating Activation Functions
 * Non-saturating activation functions refer to activation functions that do not suffer from the vanishing gradient problem to the same extent as traditional saturating activation functions like sigmoid and tanh. These functions typically have gradients that do not diminish as quickly as the __sigmoid__ or __tanh__ functions for large input values, thereby alleviating the vanishing gradient problem. 
 * One prominent example of a non-saturating activation function is the Rectified Linear Unit (__ReLU__) and its variants.

1. __Rectified Linear Unit (ReLU):__
    * ReLU is a popular activation function in deep learning, defined as __f(x)=max(0,x)__. It outputs zero for negative inputs and passes positive inputs unchanged. 
    * It's computationally efficient and doesn't suffer from vanishing gradients for positive inputs. However, it may lead to the __"dying ReLU"__ problem, causing neurons to become inactive for negative inputs. 
    * Variants like __Leaky ReLU__, __Parametric ReLU (PReLU)__, and __Exponential Linear Unit (ELU)__ address this issue by allowing a small, non-zero gradient for negative inputs.

In [4]:
# Listing ReLU variants in Keras.layers module
[m for m in dir(keras.layers) if "relu" in m.lower()]

['LeakyReLU', 'PReLU', 'ReLU', 'ThresholdedReLU']

2. __Leaky ReLU:__
    * Leaky ReLU is an activation function similar to ReLU but with a small, non-zero gradient for negative input values. 
    * It's defined as __f(x)=max(αx,x)__, where α is a small constant (e.g., 0.01). 
    * Leaky ReLU addresses the __"dying ReLU"__ problem by preventing neurons from becoming inactive for negative inputs during training. It retains the efficiency of ReLU while mitigating its limitations.

In [5]:
# Training a neural network on Fashion MNIST using the Leaky ReLU:

# Loading Fashion MNIST dataset and preprocessing
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# Setting random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Defining the neural network model with Leaky ReLU activation
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),  # Leaky ReLU activation
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),  # Leaky ReLU activation
    keras.layers.Dense(10, activation="softmax")
])

# Compiling the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

# Training the model
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 3s 1ms/step - loss: 1.2978 - accuracy: 0.6068 - val_loss: 0.8967 - val_accuracy: 0.7060
Epoch 2/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.8075 - accuracy: 0.7357 - val_loss: 0.7232 - val_accuracy: 0.7652
Epoch 3/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.6918 - accuracy: 0.7753 - val_loss: 0.6534 - val_accuracy: 0.7902
Epoch 4/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.6298 - accuracy: 0.7949 - val_loss: 0.5964 - val_accuracy: 0.8092
Epoch 5/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5894 - accuracy: 0.8072 - val_loss: 0.5635 - val_accuracy: 0.8184
Epoch 6/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5605 - accuracy: 0.8145 - val_loss: 0.5393 - val_accuracy: 0.8230
Epoch 7/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5388 - accuracy: 0.8215 - val_loss: 0.5196 - val_accuracy:

3. __Parametric ReLU:__
    * Parametric ReLU (PReLU) extends Leaky ReLU by allowing the coefficient α to be learned during training rather than fixed. PReLU can adaptively adjust the leakage for negative inputs based on data, potentially improving performance over fixed coefficients.

In [6]:
# Training a neural network on Fashion MNIST using the Leaky PReLU:

# Loading Fashion MNIST dataset and preprocessing
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# Setting random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Defining the neural network model with Leaky ReLU activation
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(10, activation="softmax")
])

# Compiling the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

# Training the model
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 3s 1ms/step - loss: 1.3262 - accuracy: 0.6042 - val_loss: 0.9065 - val_accuracy: 0.7172
Epoch 2/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.8080 - accuracy: 0.7485 - val_loss: 0.7193 - val_accuracy: 0.7750
Epoch 3/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.6862 - accuracy: 0.7824 - val_loss: 0.6488 - val_accuracy: 0.7916
Epoch 4/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.6243 - accuracy: 0.7981 - val_loss: 0.5920 - val_accuracy: 0.8072
Epoch 5/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5847 - accuracy: 0.8087 - val_loss: 0.5590 - val_accuracy: 0.8202
Epoch 6/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5563 - accuracy: 0.8159 - val_loss: 0.5349 - val_accuracy: 0.8238
Epoch 7/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5347 - accuracy: 0.8215 - val_loss: 0.5152 - val_accuracy:

4. __Exponential Linear Unit (ELU):__
    * ELU is defined as f(x)=x if x≥0 and f(x)=α(e^x−1) if x<0, where α is a small positive constant.
    * ELU behaves similarly to ReLU for positive input values but has a smooth curve for negative input values, allowing it to handle negative inputs more gracefully than ReLU. 
    * ELU also prevents the "dying ReLU" problem by ensuring a non-zero gradient for all inputs.

In [7]:
# Training a neural network on Fashion MNIST using the Leaky PReLU:

# Loading Fashion MNIST dataset and preprocessing
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# Setting random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Defining the neural network model with Leaky ReLU activation
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal", activation="elu"),
    keras.layers.Dense(100, kernel_initializer="he_normal", activation="elu"),
    keras.layers.Dense(10, activation="softmax")
])

# Compiling the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

# Training the model
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 3s 1ms/step - loss: 1.1310 - accuracy: 0.6511 - val_loss: 0.7896 - val_accuracy: 0.7516
Epoch 2/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.7255 - accuracy: 0.7628 - val_loss: 0.6542 - val_accuracy: 0.7876
Epoch 3/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.6351 - accuracy: 0.7887 - val_loss: 0.5997 - val_accuracy: 0.7978
Epoch 4/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5868 - accuracy: 0.8029 - val_loss: 0.5565 - val_accuracy: 0.8130
Epoch 5/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5557 - accuracy: 0.8117 - val_loss: 0.5314 - val_accuracy: 0.8196
Epoch 6/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5335 - accuracy: 0.8185 - val_loss: 0.5125 - val_accuracy: 0.8232
Epoch 7/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5166 - accuracy: 0.8231 - val_loss: 0.4971 - val_accuracy:

5. __Scaled Exponential Linear Unit (SELU):__
    * It's an activation function introduced to address the vanishing/exploding gradient problem while promoting self-normalizing properties in neural networks.
    * SELU has a self-normalizing property, meaning the output distribution remains close to mean 0 and standard deviation 1, which helps stabilize training.
    * It overcomes the vanishing/exploding gradient problem by promoting stable mean and variance propagation through the network.
    * It has an exponential component for negative inputs, ensuring non-zero gradients, thus avoiding the "dying ReLU" problem.
    * SELU is specifically designed for feedforward neural networks, and its performance may vary depending on the architecture and hyperparameters.

In [8]:
# Training a neural network on Fashion MNIST using the Leaky SeLU:

# Loading Fashion MNIST dataset and preprocessing
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
# Scaling pixel values to the range [0, 1]
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
# Splitting training set into validation and training subsets
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# Calculating mean and standard deviation of training pixel values
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
# Standardizing the input features using mean and standard deviation
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

# Setting random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Constructing the neural network model with SELU activation
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"))
# Adding 99 hidden layers with SELU activation
for layer in range(99):
    model.add(keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"))
# Adding output layer with softmax activation for 10 classes
model.add(keras.layers.Dense(10, activation="softmax"))

# Compiling the model with sparse categorical crossentropy loss and SGD optimizer
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

# Training the model on scaled training data for 5 epochs with validation data for monitoring
history = model.fit(X_train_scaled, y_train, epochs=5,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/5
1719/1719 [==============================] - 35s 19ms/step - loss: 1.0850 - accuracy: 0.6019 - val_loss: 0.8538 - val_accuracy: 0.6564
Epoch 2/5
1719/1719 [==============================] - 33s 19ms/step - loss: 0.7006 - accuracy: 0.7464 - val_loss: 0.6544 - val_accuracy: 0.7588
Epoch 3/5
1719/1719 [==============================] - 33s 19ms/step - loss: 0.6123 - accuracy: 0.7792 - val_loss: 0.6300 - val_accuracy: 0.7662
Epoch 4/5
1719/1719 [==============================] - 32s 18ms/step - loss: 0.5253 - accuracy: 0.8153 - val_loss: 0.4889 - val_accuracy: 0.8274
Epoch 5/5
1719/1719 [==============================] - 32s 18ms/step - loss: 0.4778 - accuracy: 0.8337 - val_loss: 0.4566 - val_accuracy: 0.8402


## Batch Normalization
 * Batch Normalization is a technique used in deep learning to enhance the training process of neural networks. 
 * This means that it adjusts the activations of each layer so that they have a mean of zero and a standard deviation of one. By doing this normalization, Batch Normalization helps to stabilize the training process and improve the speed at which neural networks converge to an optimal solution.

 * Batch Normalization has become a standard technique in deep learning because it contributes to __faster training__, __improved model performance__, and __greater stability__ during the training process.

In [9]:
# Training a neural network on Fashion MNIST using the Batch Normalization:

# Loading Fashion MNIST dataset and preprocessing
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
# Scaling pixel values to the range [0, 1]
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
# Splitting training set into validation and training subsets
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]


# Setting random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Define a Sequential model
model = keras.models.Sequential([
    # Flatten layer to convert 2D input into 1D
    keras.layers.Flatten(input_shape=[28, 28]),
    # BatchNormalization layer applied after the Flatten layer
    keras.layers.BatchNormalization(),
    # Dense layer with 300 neurons and ReLU activation function
    keras.layers.Dense(300, activation="relu"),
    # BatchNormalization layer applied after the first Dense layer
    keras.layers.BatchNormalization(),
    # Dense layer with 100 neurons and ReLU activation function
    keras.layers.Dense(100, activation="relu"),
    # BatchNormalization layer applied after the second Dense layer
    keras.layers.BatchNormalization(),
    # Dense layer with 10 neurons and softmax activation function for classification
    keras.layers.Dense(10, activation="softmax")
])

# Compiling the model with sparse categorical crossentropy loss and SGD optimizer
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

# Training the model on training data for 10 epochs with validation data for monitoring
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.8462 - accuracy: 0.7177 - val_loss: 0.5596 - val_accuracy: 0.8154
Epoch 2/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5704 - accuracy: 0.8022 - val_loss: 0.4794 - val_accuracy: 0.8402
Epoch 3/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5144 - accuracy: 0.8209 - val_loss: 0.4433 - val_accuracy: 0.8552
Epoch 4/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4780 - accuracy: 0.8334 - val_loss: 0.4224 - val_accuracy: 0.8576
Epoch 5/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4513 - accuracy: 0.8423 - val_loss: 0.4075 - val_accuracy: 0.8640
Epoch 6/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4357 - accuracy: 0.8462 - val_loss: 0.3965 - val_accuracy: 0.8680
Epoch 7/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4234 - accuracy: 0.8505 - val_loss: 0.3860 - val_accuracy:

In [10]:
# Display a summary of the model architecture
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_112 (Dense)           (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_113 (Dense)           (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)             

In [11]:
# Retrieve the BatchNormalization layer after the first layer in the model
bn1 = model.layers[1]

# List the names and trainability of the variables in the BatchNormalization layer
# This can be useful for inspecting the variables of the BatchNormalization layer
[(var.name, var.trainable) for var in bn1.variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

### Batch Normalization: Before or After Activation?
* Applying Batch Normalization (BN) __before__ the __activation function__ is a debated topic in deep learning. While sometimes it yields __better results__, it's subject to discussion. Additionally, it's common practice to omit bias terms in layers preceding BatchNormalization. Since BatchNormalization introduces its own bias parameters, setting __use_bias=False__ for these preceding layers avoids redundant parameters and optimizes model efficiency.

In [12]:
# Training a neural network on Fashion MNIST using the Batch Normalization before the activation function:

# Loading Fashion MNIST dataset and preprocessing
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
# Scaling pixel values to the range [0, 1]
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
# Splitting training set into validation and training subsets
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]


# Setting random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Define a Sequential model
model = keras.models.Sequential([
    # Flatten layer to convert 2D input into 1D
    keras.layers.Flatten(input_shape=[28, 28]),
    # BatchNormalization layer applied before the activation function
    keras.layers.BatchNormalization(),
    # Dense layer with 300 neurons and no bias term
    keras.layers.Dense(300, use_bias=False),
    # BatchNormalization layer applied before the activation function
    keras.layers.BatchNormalization(),
    # Activation function ReLU applied
    keras.layers.Activation("relu"), 
    # Dense layer with 100 neurons and no bias term
    keras.layers.Dense(100, use_bias=False),
    # BatchNormalization layer applied before the activation function
    keras.layers.BatchNormalization(),
    # Activation function ReLU applied
    keras.layers.Activation("relu"),
    # Dense layer with 10 neurons and softmax activation function for classification
    keras.layers.Dense(10, activation="softmax")
])

# Compiling the model with sparse categorical crossentropy loss and SGD optimizer
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

# Training the model on training data for 10 epochs with validation data for monitoring
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 3s 2ms/step - loss: 1.0782 - accuracy: 0.6634 - val_loss: 0.6911 - val_accuracy: 0.7852
Epoch 2/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.6859 - accuracy: 0.7807 - val_loss: 0.5655 - val_accuracy: 0.8170
Epoch 3/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.6011 - accuracy: 0.8015 - val_loss: 0.5091 - val_accuracy: 0.8334
Epoch 4/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5479 - accuracy: 0.8163 - val_loss: 0.4747 - val_accuracy: 0.8430
Epoch 5/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5158 - accuracy: 0.8252 - val_loss: 0.4524 - val_accuracy: 0.8482
Epoch 6/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4926 - accuracy: 0.8313 - val_loss: 0.4344 - val_accuracy: 0.8556
Epoch 7/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4754 - accuracy: 0.8370 - val_loss: 0.4198 - val_accuracy:

## Gradient Clipping
 * Gradient clipping addresses exploding gradients problem by imposing a constraint on the gradients during training. Specifically, it involves scaling the gradients if their norm exceeds a certain threshold.
 
 
 * __Gradient Clipping With clipvalue__: Each component of the gradient vector is individually scaled if it falls outside a specified range. This technique restricts the magnitude of each gradient component without necessarily preserving the overall direction of the gradient vector.
 
 * __Gradient Clipping With clipnorm__: The entire gradient vector is scaled down if its norm exceeds a specified threshold. This ensures that the direction of the gradient vector is preserved while preventing it from becoming too large.

  * If you want to ensure that Gradient Clipping does not change the direction of the gradient vector, you should clip by norm by setting clipnorm instead of clipvalue.

 * Gradient clipping is commonly used in recurrent neural networks (RNNs) because Batch Normalization (BN) is difficult to implement in RNNs. However, for other network types, like feedforward or convolutional neural networks (CNNs), Batch Normalization is usually sufficient.


In [13]:
# Set optimizer with gradient clipping by clipvalue
optimizer = keras.optimizers.SGD(clipvalue=1.0)

# Set optimizer with gradient clipping by clipnorm
optimizer = keras.optimizers.SGD(clipnorm=1.0)

## Reusing Pretrained Layers
* Reusing pretrained layers, often referred to as transfer learning, is a common technique in machine learning where a model trained on one task is adapted for use on a new, related task. 
* This approach is particularly effective when the pretrained model has been trained on a large dataset, as it has learned general features that can be useful for other tasks.

Let's split the fashion MNIST training set in two:
* `X_train_A`: all images of all items except for sandals and shirts (classes 5 and 6).
* `X_train_B`: a much smaller training set of just the first 200 images of sandals or shirts.

The validation set and the test set are also split this way, but without restricting the number of images.

We will train a model on set A (classification task with 8 classes), and try to reuse it to tackle set B (binary classification). We hope to transfer a little bit of knowledge from task A to task B, since classes in set A (sneakers, ankle boots, coats, t-shirts, etc.) are somewhat similar to classes in set B (sandals and shirts). However, since we are using `Dense` layers, only patterns that occur at the same location can be reused (in contrast, convolutional layers will transfer much better, since learned patterns can be detected anywhere on the image, as we will see in the CNN chapter).

In [22]:
# Training a neural network on Fashion MNIST using pretrained layers:

# Loading Fashion MNIST dataset and preprocessing
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

# Scaling pixel values to the range [0, 1]
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0

# Splitting training set into validation and training subsets
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

def split_dataset(X, y):

    # Creating masks for classes 5 (sandals) and 6 (shirts)
    y_5_or_6 = (y == 5) | (y == 6)
    # Subsetting dataset into classes A (excluding classes 5 and 6) and B (only classes 5 and 6)
    y_A = y[~y_5_or_6]
    # Adjusting class indices for class A to maintain continuity
    y_A[y_A > 6] -= 2  # class indices 7, 8, 9 should be moved to 5, 6, 7
    # For class B, creating a binary classification task: 1 for shirts (class 6), 0 otherwise
    y_B = (y[y_5_or_6] == 6).astype(np.float32)
    
    return ((X[~y_5_or_6], y_A), (X[y_5_or_6], y_B))

# Splitting datasets into subsets for classes A and B
(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)

# Limiting class B training set to 200 samples for efficiency
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]
